In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/Code/htfa_rewrite_batching/htfatorch


In [2]:
import collections
import logging
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import glob
import os
import webdataset as wds
import torch

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
affvids_dir = '/data/zulqarnain/fmri_data/AffVids_mcwm/motion_corrected/'

task_log_csvs = 'editedlogfiles'

TASK_ONSET_DELAY = 3

AFFVID_FILENAME_TEMPLATE = 'sub%02d_run%02d_mc_MNI_masked.nii.gz'

AFFVIDS_FILE = 'data/affvids2018_mc_zrest.tar'

tar_file = AFFVIDS_FILE
sink = wds.TarWriter(tar_file)


def affvid_filename(subject, run):
    return affvids_dir + (AFFVID_FILENAME_TEMPLATE % (subject, run))

class TaskElement:
    def __init__(self, task, start, end, run, fear_rating=None):
        def round_off_time(t):
            if t is not None:
                if task != 'rest':
                    return round(t + TASK_ONSET_DELAY)
                else:
                    return round(t)
            else:
                return None
        self.task = task
        self.start_time = round_off_time(start)
        self.end_time = round_off_time(end)
        self.run = run
        self.fear_rating = fear_rating

def parse_task_lines(lines):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')
        task = cols[0]
        start_time = float(cols[9])
        end_time = float(cols[10])
        run = int(cols[1])
        fear_rating = float(cols[11])
        yield TaskElement(task, start_time, end_time, run, fear_rating)

def rest_tasks(tasks):
    yield TaskElement('rest', 0, tasks[0].start_time - 1, tasks[0].run)
    for i in range(1, len(tasks)):
        rest_start = tasks[i-1].end_time + 1
        rest_end = tasks[i].start_time - 1
        if tasks[i].run == tasks[i-1].run:
            yield TaskElement('rest', rest_start, rest_end, tasks[i].run)
        else:
            yield TaskElement('rest', rest_start, None, tasks[i-1].run)
            yield TaskElement('rest', 0, rest_end, tasks[i].run)
    yield TaskElement('rest', tasks[-1].end_time + 1, None, tasks[-1].run)

bad_runs = collections.defaultdict(lambda: [])
bad_runs[3] = [1, 2, 3]
bad_runs[7] = [3]
bad_runs[14] = [3]
bad_runs[22] = [1, 2, 3]
bad_runs[23] = [1]
bad_runs[24] = [1, 2, 3]
bad_runs[26] = [3]

def read_tasks(task_csv):
    def sentinel(f):
        return f if f is not None else 0.0
    with open(task_csv, 'r') as task_csv_file:
        header = task_csv_file.readline().split(' ')
        subject = int(header[4])
        logging.info('Subject %d', subject)
        task_lines = list(parse_task_lines(task_csv_file.readlines()))
        task_lines += list(rest_tasks(task_lines))
        rest_lines = [r for r in task_lines if r.task == 'rest']
        rest_lines = sorted(rest_lines, key=lambda t: sentinel(t.run))
        rest_starts_dict = {key: [] for key in range(1, 4)}
        rest_ends_dict = {key: [] for key in range(1, 4)}
        for (i,rest) in enumerate(rest_lines):
            if rest.run in bad_runs[subject]:
                continue
            if rest.end_time is not None and rest.start_time is not None:
                rest_ends_dict[rest.run].append(rest.end_time)
                rest_starts_dict[rest.run].append(rest.start_time)
        task_lines = sorted(task_lines, key=lambda t: sentinel(t.start_time))
        for (i, task) in enumerate(task_lines):
            if task.run in bad_runs[subject]:
                logging.info('Excluding block %d %s of run %d for motion', i,
                             task.task, task.run)
                continue
            logging.info('Block %d %s of run %d started at %f, ended at %f', i,
                         task.task, task.run, sentinel(task.start_time), sentinel(task.end_time))
            result = niidb.FMriActivationBlock(zscore=True,zscore_by_rest=True)
            result.subject = subject
            result.task = task.task
            result.run = task.run
            result.block = i
            result.start_time = task.start_time
            result.end_time = task.end_time
            result.rest_start_times = rest_starts_dict[result.run]
            result.rest_end_times = rest_ends_dict[result.run]
            result.individual_differences = {'fear_rating': task.fear_rating}
            yield result

In [5]:
OVERRIDE = True

if not os.path.exists(AFFVIDS_FILE) or OVERRIDE:
    total_trs = 0
    metadata = {
        'blocks': []
    }
    block_id = 0
    for task_csv in utils.sorted_glob(affvids_dir + task_log_csvs + '/*.txt'):
        for block in read_tasks(task_csv):
            block.filename = affvid_filename(block.subject, block.run)
#             block.mask = mask_filename(block.subject, block.run)
            block.rest_end_times = '[' + ', '.join(map(str, block.rest_end_times)) + ']'
            block.rest_start_times = '[' + ', '.join(map(str, block.rest_start_times)) + ']'
            block.block = block_id
            block_id += 1
            block.mask = '/data/zulqarnain/fmri_data/AffVids_mcwm/wholebrain2.nii.gz'
            block.smooth = 6
            block.load()
            metadata['blocks'].append(block.wds_metadata())

            for vals in block.format_wds():
                sink.write(vals)
            block_trs = (block.end_time - block.start_time)
            total_trs += block_trs
        
#             affvids_db.upsert(block)
    
    metadata['voxel_locations'] = block.locations
    metadata['num_times'] = total_trs
    torch.save(metadata, tar_file + '.meta')
    logging.info('Recorded metadata, including voxel locations')

sink.close()

07/07/2021 16:26:17 Subject 10
07/07/2021 16:26:17 Block 0 rest of run 1 started at 0.000000, ended at 24.000000
07/07/2021 16:26:17 Loading Nifti image /data/zulqarnain/fmri_data/AffVids_mcwm/motion_corrected/sub10_run01_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
07/07/2021 16:26:28 Block 1 rest of run 2 started at 0.000000, ended at 22.000000
07/07/2021 16:26:28 Loading Nifti image /data/zulqarnain/fmri_data/AffVids_mcwm/motion_corrected/sub10_run02_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
07/07/2021 16:26:39 Block 2 rest of run 3 started at 0.000000, ended at 22.000000
07/07/2021 16:26:39 Loading Nifti image /data/zulqarnain/fmri_data/AffVids_mcwm/motion_corrected/sub10_run03_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_re

07/07/2021 16:27:16 Block 2 rest of run 3 started at 0.000000, ended at 23.000000
07/07/2021 16:27:16 Loading Nifti image /data/zulqarnain/fmri_data/AffVids_mcwm/motion_corrected/sub11_run03_mc_MNI_masked.nii.gz with mask /data/zulqarnain/fmri_data/AffVids_mcwm/wholebrain2.nii.gz (zscore=True, smooth=6, zscore_by_rest=True)
07/07/2021 16:27:28 Block 3 spider_high_5.m4v of run 2 started at 23.000000, ended at 44.000000
07/07/2021 16:27:28 Block 4 spider_low_4.m4v of run 1 started at 24.000000, ended at 45.000000
07/07/2021 16:27:28 Block 5 spider_low_6.m4v of run 3 started at 24.000000, ended at 45.000000
07/07/2021 16:27:28 Block 6 rest of run 2 started at 45.000000, ended at 67.000000
07/07/2021 16:27:28 Block 7 rest of run 1 started at 46.000000, ended at 68.000000
07/07/2021 16:27:28 Block 8 rest of run 3 started at 46.000000, ended at 68.000000
07/07/2021 16:27:28 Block 9 heights_low_3.mov of run 2 started at 68.000000, ended at 87.000000
07/07/2021 16:27:28 Block 10 social_low_3.m

07/07/2021 16:28:05 Block 6 rest of run 1 started at 45.000000, ended at 68.000000
07/07/2021 16:28:05 Block 7 rest of run 3 started at 45.000000, ended at 67.000000
07/07/2021 16:28:05 Block 8 rest of run 2 started at 46.000000, ended at 69.000000
07/07/2021 16:28:05 Block 9 social_high_6.mov of run 3 started at 68.000000, ended at 88.000000
07/07/2021 16:28:05 Block 10 social_low_6.m4v of run 1 started at 69.000000, ended at 88.000000
07/07/2021 16:28:05 Block 11 spider_low_5.mov of run 2 started at 70.000000, ended at 90.000000
07/07/2021 16:28:05 Block 12 rest of run 1 started at 89.000000, ended at 110.000000
07/07/2021 16:28:05 Block 13 rest of run 3 started at 89.000000, ended at 111.000000
07/07/2021 16:28:06 Block 14 rest of run 2 started at 91.000000, ended at 112.000000
07/07/2021 16:28:06 Block 15 heights_high_2.mov of run 1 started at 111.000000, ended at 132.000000
07/07/2021 16:28:06 Block 16 heights_low_3.mov of run 3 started at 112.000000, ended at 130.000000
07/07/202

07/07/2021 16:28:42 Block 13 rest of run 1 started at 89.000000, ended at 112.000000
07/07/2021 16:28:42 Block 14 rest of run 2 started at 90.000000, ended at 112.000000
07/07/2021 16:28:42 Block 15 social_low_1.m4v of run 3 started at 109.000000, ended at 130.000000
07/07/2021 16:28:43 Block 16 heights_high_4.m4v of run 1 started at 113.000000, ended at 134.000000
07/07/2021 16:28:43 Block 17 spider_low_6.m4v of run 2 started at 113.000000, ended at 133.000000
07/07/2021 16:28:43 Block 18 rest of run 3 started at 131.000000, ended at 151.000000
07/07/2021 16:28:43 Block 19 rest of run 2 started at 134.000000, ended at 157.000000
07/07/2021 16:28:43 Block 20 rest of run 1 started at 135.000000, ended at 155.000000
07/07/2021 16:28:43 Block 21 social_high_5.m4v of run 3 started at 152.000000, ended at 173.000000
07/07/2021 16:28:43 Block 22 heights_low_2.m4v of run 1 started at 156.000000, ended at 176.000000
07/07/2021 16:28:43 Block 23 spider_high_4.m4v of run 2 started at 158.000000,

07/07/2021 16:29:14 Block 24 rest of run 1 started at 173.000000, ended at 194.000000
07/07/2021 16:29:14 Excluding block 25 rest of run 3 for motion
07/07/2021 16:29:14 Block 26 rest of run 2 started at 176.000000, ended at 198.000000
07/07/2021 16:29:14 Block 27 spider_low_3.m4v of run 1 started at 195.000000, ended at 215.000000
07/07/2021 16:29:14 Excluding block 28 social_high_2.mov of run 3 for motion
07/07/2021 16:29:14 Block 29 heights_high_1.m4v of run 2 started at 199.000000, ended at 220.000000
07/07/2021 16:29:14 Block 30 rest of run 1 started at 216.000000, ended at 239.000000
07/07/2021 16:29:14 Excluding block 31 rest of run 3 for motion
07/07/2021 16:29:14 Block 32 rest of run 2 started at 221.000000, ended at 241.000000
07/07/2021 16:29:14 Block 33 heights_high_5.mov of run 1 started at 240.000000, ended at 260.000000
07/07/2021 16:29:14 Block 34 heights_low_4.m4v of run 2 started at 242.000000, ended at 263.000000
07/07/2021 16:29:14 Excluding block 35 social_high_6.m

...

07/07/2021 16:36:09 Block 10 heights_high_5.mov of run 1 started at 68.000000, ended at 88.000000
07/07/2021 16:36:09 Block 11 social_high_5.m4v of run 3 started at 68.000000, ended at 88.000000
07/07/2021 16:36:09 Block 12 rest of run 2 started at 87.000000, ended at 110.000000
07/07/2021 16:36:09 Block 13 rest of run 1 started at 89.000000, ended at 111.000000
07/07/2021 16:36:09 Block 14 rest of run 3 started at 89.000000, ended at 110.000000
07/07/2021 16:36:10 Block 15 social_low_4.m4v of run 2 started at 111.000000, ended at 132.000000
07/07/2021 16:36:10 Block 16 social_low_2.mov of run 3 started at 111.000000, ended at 132.000000
07/07/2021 16:36:10 Block 17 heights_low_6.m4v of run 1 started at 112.000000, ended at 130.000000
07/07/2021 16:36:10 Block 18 rest of run 1 started at 131.000000, ended at 154.000000
07/07/2021 16:36:10 Block 19 rest of run 2 started at 133.000000, ended at 153.000000
07/07/2021 16:36:10 Block 20 rest of run 3 started at 133.000000, ended at 155.0000

07/07/2021 16:36:47 Block 17 social_high_4_replacement.mov of run 2 started at 113.000000, ended at 133.000000
07/07/2021 16:36:47 Block 18 rest of run 1 started at 132.000000, ended at 153.000000
07/07/2021 16:36:47 Block 19 rest of run 3 started at 133.000000, ended at 156.000000
07/07/2021 16:36:47 Block 20 rest of run 2 started at 134.000000, ended at 156.000000
07/07/2021 16:36:47 Block 21 social_low_5.m4v of run 1 started at 154.000000, ended at 174.000000
07/07/2021 16:36:47 Block 22 social_high_3.mov of run 2 started at 157.000000, ended at 177.000000
07/07/2021 16:36:47 Block 23 social_low_4.m4v of run 3 started at 157.000000, ended at 178.000000
07/07/2021 16:36:47 Block 24 rest of run 1 started at 175.000000, ended at 198.000000
07/07/2021 16:36:47 Block 25 rest of run 2 started at 178.000000, ended at 199.000000
07/07/2021 16:36:47 Block 26 rest of run 3 started at 179.000000, ended at 199.000000
07/07/2021 16:36:47 Block 27 social_high_1.mov of run 1 started at 199.000000,

07/07/2021 16:37:13 Block 29 spider_low_3.m4v of run 2 started at 201.000000, ended at 222.000000
07/07/2021 16:37:13 Excluding block 30 rest of run 3 for motion
07/07/2021 16:37:13 Block 31 rest of run 1 started at 219.000000, ended at 240.000000
07/07/2021 16:37:13 Block 32 rest of run 2 started at 223.000000, ended at 243.000000
07/07/2021 16:37:13 Excluding block 33 spider_low_6.m4v of run 3 for motion
07/07/2021 16:37:13 Block 34 heights_low_1.m4v of run 1 started at 241.000000, ended at 262.000000
07/07/2021 16:37:13 Block 35 spider_low_4.m4v of run 2 started at 244.000000, ended at 265.000000
07/07/2021 16:37:13 Excluding block 36 rest of run 3 for motion
07/07/2021 16:37:13 Block 37 rest of run 1 started at 263.000000, ended at 285.000000
07/07/2021 16:37:13 Block 38 rest of run 2 started at 266.000000, ended at 287.000000
07/07/2021 16:37:13 Excluding block 39 social_low_3.m4v of run 3 for motion
07/07/2021 16:37:13 Block 40 social_low_6.m4v of run 1 started at 286.000000, end

07/07/2021 16:37:50 Block 39 spider_low_5.mov of run 1 started at 282.000000, ended at 302.000000
07/07/2021 16:37:50 Block 40 spider_high_6.mov of run 2 started at 286.000000, ended at 305.000000
07/07/2021 16:37:50 Block 41 spider_low_1.m4v of run 3 started at 289.000000, ended at 309.000000
07/07/2021 16:37:50 Block 42 rest of run 1 started at 303.000000, ended at 324.000000
07/07/2021 16:37:50 Block 43 rest of run 2 started at 306.000000, ended at 327.000000
07/07/2021 16:37:50 Block 44 rest of run 3 started at 310.000000, ended at 331.000000
07/07/2021 16:37:51 Block 45 spider_high_3.mov of run 1 started at 325.000000, ended at 345.000000
07/07/2021 16:37:51 Block 46 heights_high_2.mov of run 2 started at 328.000000, ended at 349.000000
07/07/2021 16:37:51 Block 47 heights_high_1.m4v of run 3 started at 332.000000, ended at 352.000000
07/07/2021 16:37:51 Block 48 rest of run 1 started at 346.000000, ended at 369.000000
07/07/2021 16:37:51 Block 49 rest of run 2 started at 350.0000

07/07/2021 16:38:33 Block 45 heights_high_1.m4v of run 1 started at 329.000000, ended at 349.000000
07/07/2021 16:38:33 Block 46 social_low_5.m4v of run 2 started at 329.000000, ended at 349.000000
07/07/2021 16:38:33 Block 47 spider_low_4.m4v of run 3 started at 333.000000, ended at 354.000000
07/07/2021 16:38:33 Block 48 rest of run 1 started at 350.000000, ended at 371.000000
07/07/2021 16:38:33 Block 49 rest of run 2 started at 350.000000, ended at 372.000000
07/07/2021 16:38:33 Block 50 rest of run 3 started at 355.000000, ended at 376.000000
07/07/2021 16:38:33 Block 51 social_low_6.m4v of run 1 started at 372.000000, ended at 391.000000
07/07/2021 16:38:33 Block 52 heights_low_5.m4v of run 2 started at 373.000000, ended at 393.000000
07/07/2021 16:38:33 Block 53 spider_high_1.mov of run 3 started at 377.000000, ended at 397.000000
07/07/2021 16:38:34 Block 54 rest of run 1 started at 392.000000, ended at 413.000000
07/07/2021 16:38:34 Block 55 rest of run 2 started at 394.000000

In [6]:
logging.info('Finished building NiiDb out of AffVids dataset in %s', affvids_dir)

07/07/2021 16:38:35 Finished building NiiDb out of AffVids dataset in /data/zulqarnain/fmri_data/AffVids_mcwm/motion_corrected/
